In [1]:
import pandas as pd
import numpy as np

In [10]:
df1 = pd.read_csv("brands/beer.csv")
df2 = pd.read_csv("brands/cocktail.csv")
df3 = pd.read_csv("brands/food.csv")
df4 = pd.read_csv("brands/nonAlco.csv")
df5 = pd.read_csv("brands/spirits.csv")
df6 = pd.read_csv("brands/wine.csv")

In [11]:
df1['category']='BEER'
df2['category']='COCKTAIL'
df3['category']='FOOD'
df4['category']='NONALC'
df5['category']='SPIRITS'
df6['category']='WINE'

In [14]:
df = pd.concat([df1,df2,df3,df4,df5,df6])
df

,item,brand,category
0,10 BARREL 16 BARRELS,10 BARREL,BEER
1,10 BARREL AMERICAN STOUT,10 BARREL,BEER
2,10 BARREL APOCALYPSE IPA,10 BARREL,BEER
3,10 BARREL APRICOT CRUSH,10 BARREL,BEER
4,10 BARREL BIG OL PUMPKIN,10 BARREL,BEER
...,...,...,...
63628,THIERRY GERMAIN AND MICHEL CHEVRE,THIERRY GERMAIN AND MICHEL CHEVRE,WINE
63629,CHATEAU DE L ESTANG,CHATEAU DE L ESTANG,WINE
63630,CASTILLO DE ANNA,CASTILLO DE ANNA,WINE
63631,DUCHE DE LONGUEVILLE,DUCHE DE LONGUEVILLE,WINE


In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
import re
from ftfy import fix_text

def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [25]:
import re
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
def preprocess_text(sen):
    document = re.sub(r'\\n', ' ', str(sen))
    document = re.sub(r'\\r', ' ', str(sen))
    #re.sub("\n","",line)
    
     # Remove all the special characters
    document = re.sub(r'\\n', ' ', document)
    document = re.sub(r'\W', ' ', document)
    #document = re.sub(r'\d+', ' ', document)
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z0-9]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z0-9]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    return document

In [36]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = " "  
    
    # traverse in the string   
    for ele in s: 
        ele = f'{ele}'+" "
        str1 += ele 
    
    # return string   
    return str1 

In [38]:
df

,item,brand,category
0,10 BARREL 16 BARRELS,10 BARREL,BEER
1,10 BARREL AMERICAN STOUT,10 BARREL,BEER
2,10 BARREL APOCALYPSE IPA,10 BARREL,BEER
3,10 BARREL APRICOT CRUSH,10 BARREL,BEER
4,10 BARREL BIG OL PUMPKIN,10 BARREL,BEER
...,...,...,...
63628,THIERRY GERMAIN AND MICHEL CHEVRE,THIERRY GERMAIN AND MICHEL CHEVRE,WINE
63629,CHATEAU DE L ESTANG,CHATEAU DE L ESTANG,WINE
63630,CASTILLO DE ANNA,CASTILLO DE ANNA,WINE
63631,DUCHE DE LONGUEVILLE,DUCHE DE LONGUEVILLE,WINE


In [39]:
df['prcsd_item'] = df['item'].apply(lambda com : preprocess_text(com))

In [40]:
df

,item,brand,category,prcsd_item
0,10 BARREL 16 BARRELS,10 BARREL,BEER,10 barrel 16 barrel
1,10 BARREL AMERICAN STOUT,10 BARREL,BEER,10 barrel american stout
2,10 BARREL APOCALYPSE IPA,10 BARREL,BEER,10 barrel apocalypse ipa
3,10 BARREL APRICOT CRUSH,10 BARREL,BEER,10 barrel apricot crush
4,10 BARREL BIG OL PUMPKIN,10 BARREL,BEER,10 barrel big ol pumpkin
...,...,...,...,...
63628,THIERRY GERMAIN AND MICHEL CHEVRE,THIERRY GERMAIN AND MICHEL CHEVRE,WINE,thierry germain and michel chevre
63629,CHATEAU DE L ESTANG,CHATEAU DE L ESTANG,WINE,chateau de estang
63630,CASTILLO DE ANNA,CASTILLO DE ANNA,WINE,castillo de anna
63631,DUCHE DE LONGUEVILLE,DUCHE DE LONGUEVILLE,WINE,duche de longueville


In [41]:
dft = pd.read_csv("TestFile.csv")

In [44]:
dft['filtrd_item'] = dft['Item Name'].apply(lambda x: preprocess_text(x))

In [47]:
df['prcsd_item'] = df['prcsd_item'].apply(lambda x: x.strip())

In [49]:
dft['filtrd_item'] = dft['filtrd_item'].apply(lambda x: x.strip())

In [71]:
df = df[(df['prcsd_item'].str.len()>1) & ~(df['prcsd_item'].isnull())]

In [72]:
dft = dft[(dft['filtrd_item'].str.len()>1) & ~(dft['filtrd_item'].isnull())]

In [73]:
ln_comb_chk = []
from sklearn.neighbors import NearestNeighbors
import time
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices
matches = []
#['category_food', 'category_beer', 'alcoholicbeverages', 'category_other', 'category_spirits', 'category_wine']
d = {'category_df':df[['prcsd_item']],'category_dft':dft[['filtrd_item']]}
for k, my_df1 in d.items():
    for l, my_df2 in d.items():
        ln_comb = l + '/' + k
        if ((k != l) & ~(ln_comb in ln_comb_chk)):
            ln_comb_chk.append(k + '/' + l)
            org_name_clean = my_df1['prcsd_item'].unique()
            print('Vecorizing the data - this could take a few minutes for large datasets...'+k+'/'+l)
            vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
            tfidf = vectorizer.fit_transform(org_name_clean)
            print('Vecorizing completed...')
            nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
            org_column = 'filtrd_item'
            unique_org = my_df2[org_column].values
            t1 = time.time()
            distances, indices = getNearestN(unique_org)
            t = time.time()-t1
            print("COMPLETED IN:", t)
            unique_org = list(unique_org)
            for i,j in enumerate(indices):
                temp = [round(distances[i][0],2), listToString(org_name_clean[j]),unique_org[i],k,l]
                matches.append(temp)
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Origional name','Matched_In','Originally_In'])

Vecorizing the data - this could take a few minutes for large datasets...category_df/category_dft
Vecorizing completed...
COMPLETED IN: 3.947706699371338


In [74]:
matches

,Match confidence (lower is better),Matched name,Origional name,Matched_In,Originally_In
0,0.89,mango rickey,mango rita,category_df,category_dft
1,0.80,bud light lime rita,lime rita,category_df,category_dft
2,0.76,chowder clam,cup white clam chowder,category_df,category_dft
3,0.82,open beer,open cater,category_df,category_dft
4,0.59,panini pepperoni,pepperoni pz,category_df,category_dft
...,...,...,...,...,...
623,0.70,nigiri sake,fresh sake nigiri,category_df,category_dft
624,1.05,fish filet,wine dine filet,category_df,category_dft
625,1.17,fish filet,opt4 6oz port wine filet,category_df,category_dft
626,0.87,crimson spice champagne cocktail,treno champagne cocktail,category_df,category_dft
